In [ ]:
# !pip install aiomoex
# !pip install tinkoff-investments
# !pip install yfinance

Открытые вопросы:
* Стоит ли описывать варианты получения данных и причину выбора?
* Идея - разбивать на 2 интервал. До СВО и после, что-то вроде микро исследования. Периоды для обучения - 2-3 года. Прогноз (в идеале) как в исследовании - 30 минутные интервалы с дообучением (ноль понимания как его делать в реалтайм)


Что нужно сделать ближайшее:
* Выбрать инструменты для результирующего сравнения своей стратегии с HODL
* Выбрать инструменты для прогнозирования: металлы, фонды, акции, сырье. Не более 10, желательно даже 5
* Выгрузить IMOEX индекс на периоде для сравнения (в идеале потом можно еще с открытием вкладов сравнить, но сложно кодом считать)
* Реализовать функции расчета индикаторов, которые необходимы будут для обучения


In [3]:
from tinkoff.invest import Client, AsyncClient, CandleInterval, SecurityTradingStatus
from tinkoff.invest.services import InstrumentsService
from tinkoff.invest.utils import quotation_to_decimal, now
from tinkoff.invest.caching.instruments_cache.instruments_cache import InstrumentsCache

import pandas as pd
from dotenv import load_dotenv

import os
# import asyncio
import time
from datetime import datetime, timezone, timedelta
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

In [4]:
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

In [15]:
load_dotenv()

TOKEN = "t.eRZ0sm51xUBVmE4xcdx4cebt0iTp8oIbMUFhdf2Ot7GOG7v0BjTryycaTnwiuyJbwpiqpLnKK6WNMCSdHiPZlg"
# os.getenv("TOKEN")

Подробная информация по тикеру.

**Не работает с индексами вроде индекса МосБиржи**

In [6]:
def get_info(ticker, TOKEN = TOKEN):
    with Client(TOKEN) as client:
        instruments: InstrumentsService = client.instruments
        tickers = []
        for method in ["shares", "bonds", "etfs", "currencies", "futures"]:
            for item in getattr(instruments, method)().instruments:
                tickers.append(
                    {
                        "name": item.name,
                        "ticker": item.ticker,
                        "class_code": item.class_code,
                        "figi": item.figi,
                        "uid": item.uid,
                        "type": method,
                        "min_price_increment": quotation_to_decimal(
                            item.min_price_increment
                        ),
                        "scale": 9 - len(str(item.min_price_increment.nano)) + 1,
                        "lot": item.lot,
                        "trading_status": str(
                            SecurityTradingStatus(item.trading_status).name
                        ),
                        "api_trade_available_flag": item.api_trade_available_flag,
                        "currency": item.currency,
                        "exchange": item.exchange,
                        "buy_available_flag": item.buy_available_flag,
                        "sell_available_flag": item.sell_available_flag,
                        "short_enabled_flag": item.short_enabled_flag,
                        "klong": quotation_to_decimal(item.klong),
                        "kshort": quotation_to_decimal(item.kshort),
                    }
                )

        tickers_df = pd.DataFrame(tickers)

        ticker_df = tickers_df[tickers_df["ticker"] == ticker]
        if ticker_df.empty:
            print("There is no such ticker: %s", ticker)
            # return

        figi = ticker_df["figi"].iloc[0]
        print(f"\nTicker {ticker} have figi={figi}\n")
        print(f"Additional info for this {ticker} ticker:")
        print(ticker_df.iloc[0])

        return figi

ticker = "SPY"
get_info(ticker)


Ticker SPY have figi=BBG000BDTBL9

Additional info for this SPY ticker:
name                                          SPDR S&P 500 ETF Trust
ticker                                                           SPY
class_code                                                     SPBXM
figi                                                    BBG000BDTBL9
uid                             59ba0c48-4f13-429a-b1a1-6aff71baca05
type                                                            etfs
min_price_increment                                       0.01000000
scale                                                              2
lot                                                                1
trading_status              SECURITY_TRADING_STATUS_BREAK_IN_TRADING
api_trade_available_flag                                        True
currency                                                         usd
exchange                                                     unknown
buy_available_flag            

'BBG000BDTBL9'

Получить только FIGI.

In [7]:
def get_figi(ticker, TOKEN = TOKEN):
    with Client(TOKEN) as client:
        instruments: InstrumentsService = client.instruments
        for method in ["shares", "bonds", "etfs", "currencies", "futures"]:
            for item in getattr(instruments, method)().instruments:
                return item.figi

ticker = "SPY"
get_figi(ticker)

'BBG000BBCQD7'

In [8]:
def get_5min(ticker: str) -> pd.DataFrame:
    ticker = ticker.upper()
    file_path = DATA_DIR / f"{ticker}_5min.parquet"

    # Если файл есть — читаем и сразу делаем DateTime колонкой
    if file_path.exists():
        print(f"Загружаю из кэша: {ticker}")
        df = pd.read_parquet(file_path)
        if df.index.name == "DateTime":
            df = df.reset_index()
        return df.sort_values("DateTime", ascending=False)

    # Если файла нет — скачиваем
    print(f"Скачиваю {ticker} с 01.01.2025...")
    figi = get_figi(ticker)
    new_rows = []
    current = datetime(2025, 1, 1)
    now = datetime.now()

    with Client(TOKEN) as client:
        while current < now:
            end = min(current + timedelta(days=7), now - timedelta(minutes=10))

            if end <= current:
                break

            for _ in range(3):
                try:
                    candles = client.market_data.get_candles(
                        figi=figi,
                        from_=current,
                        to=end,
                        interval=CandleInterval.CANDLE_INTERVAL_5_MIN
                    ).candles

                    for c in candles:
                        new_rows.append({
                            "DateTime": c.time,
                            "Open":  c.open.units  + c.open.nano  / 1e9,
                            "High":  c.high.units  + c.high.nano  / 1e9,
                            "Low":   c.low.units   + c.low.nano   / 1e9,
                            "Close": c.close.units + c.close.nano / 1e9,
                            "Volume": c.volume
                        })
                    print(f"  +{len(candles)} свечей → {end.strftime('%Y-%m-%d %H:%M')}")
                    break
                except Exception as e:
                    if "RESOURCE_EXHAUSTED" in str(e):
                        print("  Лимит! Жду 45 сек...")
                        time.sleep(45)
                    else:
                        time.sleep(5)
            else:
                print(f"  Не удалось скачать кусок {current.date()} → {end.date()}")
            
            current = end
            time.sleep(1.2)

    if not new_rows:
        raise RuntimeError(f"Не удалось скачать данные для {ticker}")

    df = pd.DataFrame(new_rows)
    df = df.sort_values("DateTime", ascending=False)
    
    df.to_parquet(file_path, compression="zstd", index=False)
    print(f"Сохранено: {file_path} → {len(df):,} строк")

    return df

In [ ]:
# df.head(10)

NameError: name 'df' is not defined

Капитализация = Цена закрытия × Количество акций в свободном обращении (free-float)

In [33]:
import yfinance as yf

stock = yf.Ticker('SBER.ME')
stock.info
# tickers = ['SBER.ME', 'GAZP.ME', 'ROSN.ME', 'NVTK.ME', 'GMKN.ME',
#            'LKOH.ME', 'PLZL.ME', 'YNDX.ME', 'SNGS.ME', 'SNGSP.ME']  # Пример тикеров для MOEX

# data = {}
# for ticker in tickers:
#     stock = yf.Ticker(ticker)
#     info = stock.info
#     data[ticker] = info.get('marketCap', None)

# # Отсортировать по капитализации
# # sorted_tickers = sorted(data.items(), key=lambda x: x[1], reverse=True)
# for ticker, cap in data.items():
#     print(f"{ticker}: {cap}")

{'address1': '19 Vavilova Street',
 'city': 'Moscow',
 'zip': '117312',
 'country': 'Russia',
 'phone': '7 495 957 5731',
 'fax': '7 495 747 3731',
 'website': 'https://www.sberbank.ru',
 'industry': 'Banks - Regional',
 'industryKey': 'banks-regional',
 'industryDisp': 'Banks - Regional',
 'sector': 'Financial Services',
 'sectorKey': 'financial-services',
 'sectorDisp': 'Financial Services',
 'longBusinessSummary': 'Sberbank of Russia, together with its subsidiaries, engages in corporate and retail banking to corporate customers, large, medium, and small businesses, government bodies, and financial organizations in the Russian Federation. It operates through B2B (business to business); B2C (business to clients): Other segments.The company offers online banking; deposit and corporate structured products; ending products to corporate clients comprising lending, trade and export financing, interbank and overdraft lending, REPO, leasing, and other financing instruments; lending products 